In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

if not tf.__version__.startswith('2'):
    raise ValueError('This code requires TensorFlow V2.x')

In [4]:
(train_data, test_data), info = tfds.load('imdb_reviews',split=[tfds.Split.TRAIN, tfds.Split.TEST],as_supervised=True,with_info=True)
info

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

ConnectionError: HTTPConnectionPool(host='ai.stanford.edu', port=80): Max retries exceeded with url: /~amaas/data/sentiment/aclImdb_v1.tar.gz (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x7f0cc05130d0>: Failed to resolve 'ai.stanford.edu' ([Errno -3] Temporary failure in name resolution)"))

In [3]:
print(len(train_data))
print(len(test_data))

NameError: name 'train_data' is not defined

In [ ]:
for review, label in train_data.take(2):  
    print("Review:", review.numpy().decode('utf-8'))  
    print("Sentiment:", "Positive" if label.numpy() == 1 else "Negative")

In [ ]:
# Tokenize and pad the reviews
tokenizer = keras.preprocessing.text.Tokenizer(num_words=10000, oov_token='<OOV>')
train_sentences, train_labels = zip(*[(sent.numpy().decode('utf8'), label.numpy()) for sent, label in train_data])
tokenizer.fit_on_texts(train_sentences)

train_sequences = tokenizer.texts_to_sequences(train_sentences)
train_padded = keras.preprocessing.sequence.pad_sequences(train_sequences, maxlen=100, padding='post', truncating='post')

test_sentences, test_labels = zip(*[(sent.numpy().decode('utf8'), label.numpy()) for sent, label in test_data])
test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded = keras.preprocessing.sequence.pad_sequences(test_sequences, maxlen=100, padding='post', truncating='post')

train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [ ]:
model = keras.models.Sequential([
            keras.layers.Embedding(10000, 64),
            keras.layers.SimpleRNN(64),
            keras.layers.Dense(1, activation='sigmoid')
        ])

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

history = model.fit(train_padded, train_labels, epochs=1, validation_data=(test_padded, test_labels))

In [ ]:
print("Training Loss per epoch:", history.history['loss'])
print("Training Accuracy per epoch:", history.history['accuracy'])
print("Validation Loss per epoch:", history.history['val_loss'])
print("Validation Accuracy per epoch:", history.history['val_accuracy'])

In [ ]:
test_loss, test_acc = model.evaluate(test_padded, test_labels)
print(f"Test Accuracy: {test_acc:.4f}")
print(f"Test Loss: {test_loss:.4f}")

In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Model Accuracy')
plt.show()

# Plot training & validation loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Model Loss')
plt.show()

In [ ]:
sample_texts = ["This is a great product!", "I had a terrible experience."]
sample_sequences = tokenizer.texts_to_sequences(sample_texts)
sample_padded = keras.preprocessing.sequence.pad_sequences(sample_sequences, maxlen=100, padding='post', truncating='post')

predictions = model.predict(sample_padded)
predicted_labels = ['Positive' if p > 0.5 else 'Negative' for p in predictions]

for text, label in zip(sample_texts, predicted_labels):
    print(f"Review: {text} | Sentiment: {label}")

In [ ]:
model = keras.models.Sequential([
            keras.layers.Embedding(10000, 32), # Reduced embedding dimension
            keras.layers.Dropout(0.5), # Added dropout
            keras.layers.SimpleRNN(32, kernel_regularizer=keras.regularizers.l2(0.001)), # L2 regularization
            keras.layers.Dense(1, activation='sigmoid')
        ])

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Adding early stopping callback
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

history = model.fit(train_padded, train_labels, epochs=1, validation_data=(test_padded, test_labels), callbacks=[early_stop])

In [ ]:
test_loss, test_acc = model.evaluate(test_padded, test_labels)
print(f"Test Accuracy: {test_acc:.4f}")
print(f"Test Loss: {test_loss:.4f}")

In [ ]:
sample_texts = ["This is a great product!", "I had a terrible experience."]
sample_sequences = tokenizer.texts_to_sequences(sample_texts)
sample_padded = keras.preprocessing.sequence.pad_sequences(sample_sequences, maxlen=100, padding='post', truncating='post')

predictions = model.predict(sample_padded)
predicted_labels = ['Positive' if p > 0.5 else 'Negative' for p in predictions]

for text, label in zip(sample_texts, predicted_labels):
    print(f"Review: {text} | Sentiment: {label}")

In [ ]:
from keras.layers import Bidirectional, LSTM

model = keras.models.Sequential([
            keras.layers.Embedding(10000, 32),
            Bidirectional(LSTM(32, dropout=0.5, recurrent_dropout=0.5, kernel_regularizer=keras.regularizers.l2(0.001))),
            keras.layers.Dense(1, activation='sigmoid')
        ])

model.compile(loss="binary_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=0.001), metrics=["accuracy"])

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

history = model.fit(train_padded, train_labels, epochs=1, validation_data=(test_padded, test_labels), callbacks=[early_stop])

In [ ]:
test_loss, test_acc = model.evaluate(test_padded, test_labels)
print(f"Test Accuracy: {test_acc:.4f}")
print(f"Test Loss: {test_loss:.4f}")

In [ ]:
sample_texts = ["This is a great product!", "I had a terrible experience."]
sample_sequences = tokenizer.texts_to_sequences(sample_texts)
sample_padded = keras.preprocessing.sequence.pad_sequences(sample_sequences, maxlen=100, padding='post', truncating='post')

predictions = model.predict(sample_padded)
predicted_labels = ['Positive' if p > 0.5 else 'Negative' for p in predictions]

for text, label in zip(sample_texts, predicted_labels):
    print(f"Review: {text} | Sentiment: {label}")

In [ ]:
from keras.layers import Conv1D, MaxPooling1D

model = keras.models.Sequential([
            keras.layers.Embedding(10000, 32),
            Conv1D(32, 7, activation='relu'),
            MaxPooling1D(5),
            Bidirectional(LSTM(32, return_sequences=True, dropout=0.5, recurrent_dropout=0.5, kernel_regularizer=keras.regularizers.l2(0.001))),
            Bidirectional(LSTM(32, dropout=0.5, recurrent_dropout=0.5, kernel_regularizer=keras.regularizers.l2(0.001))),
            keras.layers.Dense(1, activation='sigmoid')
        ])

model.compile(loss="binary_crossentropy", optimizer=keras.optimizers.RMSprop(learning_rate=0.001), metrics=["accuracy"])

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
checkpoint = keras.callbacks.ModelCheckpoint("best_model.keras", save_best_only=True)

history = model.fit(train_padded, train_labels, epochs=1, validation_data=(test_padded, test_labels), callbacks=[early_stop, checkpoint])

In [ ]:
test_loss, test_acc = model.evaluate(test_padded, test_labels)
print(f"Test Accuracy: {test_acc:.4f}")
print(f"Test Loss: {test_loss:.4f}")

In [ ]:
sample_texts = ["This is a great product!", "I had a terrible experience."]
sample_sequences = tokenizer.texts_to_sequences(sample_texts)
sample_padded = keras.preprocessing.sequence.pad_sequences(sample_sequences, maxlen=100, padding='post', truncating='post')

predictions = model.predict(sample_padded)
predicted_labels = ['Positive' if p > 0.5 else 'Negative' for p in predictions]

for text, label in zip(sample_texts, predicted_labels):
    print(f"Review: {text} | Sentiment: {label}")